In [1]:
import sys
from pathlib import Path

# Current notebook location
notebook_path = Path().resolve()

# Add parent folder (meta/) to sys.path
sys.path.append(str(notebook_path.parent))

In [2]:
import os
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from preprocess.preprocess_final import preprocess_pipeline
from utils.padding_batch_reg import collate_batch
from add_ons.feature_pipeline_base import FeaturePipeline
from utils.generate_name import generate_filenames
from models.neural_nets.vanilla_fnn import VanillaFNN
from models.evaluation.multi_regression import evaluate_model
from models.utils.early_stopping import get_early_stopping_callbacks
from utils.run_debug_mode import run_debug_mode
from utils.save_model_files import save_model_files
from sequencer.sequencer import SequencerAddOn
from add_ons.feature_tracker_addon import FeatureColumnTrackerAddOn
from add_ons.label_padder_add_on import LabelPadder
from add_ons.input_dim_calculator import InputDimCalculator
from add_ons.candle_norm_reduce_addon import CandleNormalizationAddOn
from add_ons.candle_shape_add_on import CandleShapeFeaturesAddOn
from add_ons.drop_column_windowing_add_on import DropColumnsAddOn
from add_ons.real_price_multiplier import RealPriceMultiplier
from utils.filter_sequences import FilterInvalidSequencesAddOn
from add_ons.prepare_output import PrepareOutputAddOn
from add_ons.RootPower import RootPowerMapperAddOn
from add_ons.ArcTanMapper import ArctanMapperAddOn

def train_model(
    data_csv,
    labels_csv,
    experiment_out_dir="experiments",
    do_validation=True,
    hidden_dim=10,
    lr=0.0001,
    batch_size=50,
    max_epochs=200,
    save_model=True,
    return_val_accuracy = True,
    test_mode = False,
    early_stop = False,
    optimizer_name= "adamw",
    scheduler_name = "reduce_on_plateau",
    optimizer_params={"weight_decay": 0.01},
    scheduler_params={"factor": 0.2, "patience": 5},
    activation_function = "relu",
    use_mse_loss = False,
    use_rescue = False,
    activation_functions = ["relu"]
):
    
    # 2. Create the pipeline and add your modules
    feature_pipeline = FeaturePipeline(
        add_ons=[
            SequencerAddOn(include_cols=None, exclude_cols=None),
            CandleNormalizationAddOn(),
            ArctanMapperAddOn(
                a=3,
                  b=1,  # Use a root power for aggressive variance increase
            target_features={"main":["open_prop", "high_prop", "low_prop", "close_prop"]}, # Apply to features
            y=True,
        ),
            DropColumnsAddOn(cols_map={ "main": ["open", "high", "low", "close", "volume"]}),
            FilterInvalidSequencesAddOn(),
            LabelPadder(),
            FeatureColumnTrackerAddOn(), 
            InputDimCalculator(),
            PrepareOutputAddOn(metadata_keys=["last_close_price"]),
            RealPriceMultiplier()
        ])
    model_save_path, meta_save_path, pipeline_save_path, folder_name = generate_filenames([
        ("model", "pt"),
        ("meta", "pkl"),
        ("pipeline", "pkl"),
        "fnn"
    ])
    # feature_pipeline.method_table()
    # Preprocess: pad linePrices and sequences
    if do_validation:
        train_ds, val_ds, returned_state = preprocess_pipeline(
            data_csv, labels_csv,
            val_split=True,
            debug_indices=[0,1,2],
            feature_pipeline=feature_pipeline,
        )
    else:
        train_ds, returned_state = preprocess_pipeline(
            data_csv, labels_csv,
            val_split=True,
            debug_indices=[0,1,2],
            feature_pipeline=feature_pipeline,
        )
        val_ds = None

    input_dim = returned_state['input_dim']
    max_len_y = returned_state['max_len_y']
    feature_columns = returned_state["feature_columns"]
    print("feature_columns",feature_columns)
    model = VanillaFNN(
        input_dim=input_dim,
        hidden_dim=hidden_dim,
        output_dim=max_len_y,
        lr=lr,
        optimizer_name= optimizer_name,
        scheduler_name = scheduler_name,
        optimizer_params= optimizer_params,
        scheduler_params= scheduler_params ,
        use_mse_loss = use_mse_loss,
        activation_functions = activation_functions,
        use_rescue = use_rescue
    )

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=False, collate_fn=collate_batch)
    val_loader = DataLoader(val_ds, batch_size=batch_size, collate_fn=collate_batch) if val_ds else None
    if early_stop:
        # Define the output directory for this specific experiment/model
        model_out_dir = os.path.join(experiment_out_dir, folder_name)
        os.makedirs(model_out_dir, exist_ok=True)
        callbacks = get_early_stopping_callbacks(model_out_dir) if early_stop else []

    trainer = pl.Trainer(
        max_epochs=max_epochs,
        accelerator="auto",
        log_every_n_steps= 3,
        devices=1,
        fast_dev_run=test_mode,
        gradient_clip_val=1.0,
        gradient_clip_algorithm="norm",
        callbacks= callbacks if early_stop else None
    )

    trainer.fit(model, train_loader, val_loader)

    # --- Debug / Test mode --- #
    if test_mode:
        save_model, df_seq = run_debug_mode(train_loader, feature_columns, test_mode)

    if save_model:
        save_model_files(
            base_dir=experiment_out_dir,
            folder_name=folder_name,
            trainer=trainer,
            model_out=model_save_path,
            meta_out=meta_save_path,
            pipeline_out=pipeline_save_path,
            pipeline=feature_pipeline,
            train_model=train_model,
        )

    # --- Evaluation --- #
    if do_validation:
        metrics = evaluate_model(model, val_loader, feature_pipeline)
        if return_val_accuracy:
            return {"accuracy": metrics["mse"] * (-1)}
        
if __name__ == "__main__":
    train_model(
        "/home/iatell/projects/meta-learning/data/Bitcoin_BTCUSDT_kaggle_1D_candles.csv",
        "/home/iatell/projects/meta-learning/data/baseline_regression.csv",
        do_validation=True,
        test_mode = True,
        max_epochs=200,
        hidden_dim=100,
        lr=0.001,
        batch_size=50,
        optimizer_name= "adamw",
        scheduler_name = None,
        optimizer_params={},
        scheduler_params={},
        save_model= False,
        use_mse_loss = True,
        use_rescue = False,
       activation_functions=["elu", 
                            #  "elu", "dropout(0.1)", "elu"
                             ] #"leaky_relu" "sigmoid" "tanh"  "elu" "relu" "swish" "mish"
    )


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.


✅ Sequencer returned 78 aligned sequences in SequenceCollection.
✅ Padded 78 label sequences to shape: (78, 9)
✅ Calculated input_dim: {'main': 4}

=== DEBUG SAMPLE CHECK ===

--- Sample 0 ---
Original index: 0
Window range: [not available]
y (shape (9,)): [ 0.23053596 -0.79516724 -0.79516724 -0.79516724 -0.79516724 -0.79516724
 -0.79516724 -0.79516724 -0.79516724]
Feature sets:
  • 'main' → DataFrame (3, 4)
    Columns: ['open_prop', 'high_prop', 'low_prop', 'close_prop']
    Preview:
  open_prop  high_prop  low_prop  close_prop
  0.229527   0.301455  0.110293    0.163807
  0.163942   0.236210 -0.015501    0.230101
  0.234245   0.237839 -0.251339    0.000000

--- Sample 1 ---
Original index: 1
Window range: [not available]
y (shape (9,)): [ 0.39999014 -0.79516724 -0.79516724 -0.79516724 -0.79516724 -0.79516724
 -0.79516724 -0.79516724 -0.79516724]
Feature sets:
  • 'main' → DataFrame (3, 4)
    Columns: ['open_prop', 'high_prop', 'low_prop', 'close_prop']
    Preview:
  open_prop  hig

You are using a CUDA device ('NVIDIA GeForce RTX 3060') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params | Mode 
-----------------------------------------------
0 | layers  | Sequential | 2.2 K  | train
1 | flatten | Flatten    | 0      | train
2 | loss_fn | L1Loss     | 0      | train
-----------------------------------------------
2.2 K     Trainable params
0         Non-trainable params
2.2 K     Total params
0.009     Total estimated model params size (MB)
6         Modules in train mode
0         Modules in eval mode
/home/iatell/envs/Rllib2.43/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'train_dataload

Training: |          | 0/? [00:00<?, ?it/s]

X {'main': tensor([[[ 3.8798e-02,  6.5966e-02, -2.5274e-02,  4.6991e-03],
         [ 4.9695e-03,  2.3351e-02, -1.4256e-02,  1.4746e-02],
         [ 1.4794e-02,  1.5481e-02, -7.4300e-03,  0.0000e+00]],

        [[ 6.9479e-02,  1.0852e-01, -1.0525e-03,  5.0485e-02],
         [ 5.0485e-02,  7.6393e-02,  2.1328e-02,  5.6683e-02],
         [ 5.6644e-02,  6.7805e-02, -4.1505e-02,  0.0000e+00]],

        [[ 1.7609e-01,  1.8077e-01, -6.7597e-02, -2.8602e-02],
         [-2.8317e-02,  3.8241e-02, -3.9934e-02,  4.0487e-03],
         [ 4.0487e-03,  3.8179e-02, -1.1902e-01,  0.0000e+00]],

        [[ 2.6483e-01,  2.8301e-01,  1.6832e-01,  1.8465e-01],
         [ 1.8465e-01,  2.5912e-01,  1.6782e-01,  2.2197e-01],
         [ 2.2197e-01,  2.2807e-01, -4.0091e-02,  0.0000e+00]],

        [[ 8.0669e-02,  1.1356e-01,  3.3766e-02,  6.2449e-02],
         [ 6.2096e-02,  9.1314e-02,  3.9795e-02,  8.4277e-02],
         [ 8.4277e-02,  9.0075e-02, -1.9993e-02,  0.0000e+00]],

        [[ 8.1059e-02,  8.1059e-02

Validation: |          | 0/? [00:00<?, ?it/s]

/home/iatell/envs/Rllib2.43/lib/python3.11/site-packages/pytorch_lightning/utilities/data.py:79: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 16. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
`Trainer.fit` stopped: `max_steps=1` reached.


🔍 Debug batch:
  Keys in X_batch: ['main']
  y_batch shape: torch.Size([50, 9])
  First label in batch: tensor([ 0.0137, -0.7952, -0.7952, -0.7952, -0.7952, -0.7952, -0.7952, -0.7952,
        -0.7952])

Feature group: main
  X_batch shape: torch.Size([50, 3, 4])
  First sequence in batch (first steps):
 tensor([[ 0.0388,  0.0660, -0.0253,  0.0047],
        [ 0.0050,  0.0234, -0.0143,  0.0147],
        [ 0.0148,  0.0155, -0.0074,  0.0000]])

✅ Combined df_seq shape: (150, 4)
✅ Column names in df_seq: ['open_prop', 'high_prop', 'low_prop', 'close_prop']
ArctanMapperAddOn: Inverse-transforming labels/predictions with A=3.0000, B=1.0000...
➡️ Running RealPriceMultiplier on_evaluation...
  ✅ RealPriceMultiplier: Scaled predictions and labels to real prices.

📊 Validation Metrics (Hungarian matched):
  Regression → MSE: 440414848.000000, MAE: 15214.421875, MAE/LastClose: 0.882894


In [9]:
df_seq

,open_prop,high_prop,low_prop,close_prop
0,0.142618,0.186183,-0.115067,0.049603
1,0.051011,0.110600,-0.086403,0.087876
2,0.088019,0.090043,-0.062374,0.000000
3,0.191113,0.239535,-0.023476,0.162756
4,0.162756,0.200481,0.105695,0.172504
...,...,...,...,...
145,0.163202,0.196745,-0.101741,0.171715
146,0.171716,0.202343,-0.153074,0.000000
147,0.151474,0.218506,-0.103479,0.091243
148,0.091243,0.213446,-0.019882,0.111623
